In [576]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression

le = LabelEncoder()

In [577]:
database = pd.read_csv('COBRA-YTD2017.csv')

In [578]:
database.rename(columns={'UC2 Literal' : 'Crime', 'occur_time': 'time', 'MaxOfnum_victims': 'victims', 'Avg Day': 'day'}, inplace=True)
# database.head()

In [579]:
# NOTE : I removed rows where victims are only 1, it screws up the results.
# database.day.unique()
arr = [23, 24, 21, 19, 27, 17, 15]
database = database[database.victims != 23]
database = database[database.victims != 24]
database = database[database.victims != 21]
database = database[database.victims != 19]
database = database[database.victims != 27]
database = database[database.victims != 17]
database = database[database.victims != 15]
# database = database[database.Crime != 'LARCENY-FROM VEHICLE']
array = database.Crime.unique()

In [594]:
# NOTE: Clean up database and remove strings basically.


# NOTE: Crime column.
array = le.fit((np.array(database.Crime.unique())).tolist())
database.Crime = le.transform((np.array(database.Crime)).tolist())

# NOTE: Day column
le.fit(np.array(database.day.unique()).tolist())
database.day = le.transform((np.array(database.day)).tolist())

# NOTE : Neighborhood
le.fit(np.array(database.neighborhood.unique()).tolist())
database.neighborhood = le.transform(np.array(database.neighborhood).tolist())

# NOTE : Extract hour
database['hour'] = pd.to_datetime(database.time).dt.hour

# NOTE : Extract month
database['month'] = pd.to_datetime(database.occur_date).dt.month

# NOTE : Extract day
database['days'] = pd.to_datetime(database.occur_date).dt.day

# Take relevant data to different DataFrame
df = database[['Crime', 'day', 'days', 'hour', 'victims', 'neighborhood', 'x', 'y', 'month']]

# Make dummy variables
df = df.join(pd.get_dummies(df.hour, prefix="hour"))
df = df.join(pd.get_dummies(df.days, prefix="days"))
df = df.join(pd.get_dummies(df.month, prefix="month"))
df = df.join(pd.get_dummies(df.day, prefix="day"))


# df.head()

,Crime,day,days,hour,victims,neighborhood,x,y,month,hour_0.0,...,month_11,month_12,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,5,2,30,23.0,2.0,78,-84.38013,33.75582,12,0,...,0,1,0,0,1,0,0,0,0,0
1,5,6,18,13.0,1.0,115,-84.39745,33.78674,12,0,...,0,1,0,0,0,0,0,0,1,0
2,5,2,30,22.0,1.0,143,-84.39486,33.73760,12,0,...,0,1,0,0,1,0,0,0,0,0
3,5,2,30,20.0,1.0,62,-84.39887,33.75156,12,0,...,0,1,0,0,1,0,0,0,0,0
4,5,3,31,0.0,2.0,1,-84.46522,33.72146,12,1,...,0,1,0,0,0,1,0,0,0,0


In [602]:
X_all = df.drop(['Crime'], 1)
y_all = df['Crime']

X_all = X_all.fillna(0)

In [604]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state = 2)

In [608]:
classif = OneVsRestClassifier(SVC(kernel = 'rbf'))
# classif.fit(X_train, y_train)

# classif = RandomForestClassifier()
# classif = DecisionTreeClassifier(max_depth = 3)
# classif = LinearRegression()
classif.fit(X_train, y_train)

y_pred = classif.predict(X_train)

print(sum(y_train == y_pred) / float(len(y_pred)))
# print(precision_score(y_train, y_pred, average='micro'))
# print(recall_score(y_train, y_pred, average='weighted'))
# print(f1_score(y_train, y_pred, average='macro'))

y_pred = classif.predict(X_test)
# print(y_pred)

print(sum(y_test == y_pred) / float(len(y_pred)))
# print(precision_score(y_test, y_pred, average='micro'))
# print(f1_score(y_test, y_pred, average='micro'))

0.6938795853269537
0.38621411483253587
